In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/climate-change-edsa2020-21/train.csv
/kaggle/input/climate-change-edsa2020-21/test.csv
/kaggle/input/climate-change-edsa2020-21/sample_submission.csv


Importing libraries

In [2]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.utils import resample
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import f1_score

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Loading data

In [3]:
train = pd.read_csv('../input/climate-change-edsa2020-21/train.csv')
test = pd.read_csv('../input/climate-change-edsa2020-21/test.csv')

Balancing the data

In [4]:
believe = train[train['sentiment'] ==1]
no_belief = train[train['sentiment'] == -1]
neutral = train[train['sentiment'] == 0]
news = train[train['sentiment'] == 2]

In [5]:
no_belief_upsampled = resample(no_belief, replace=True, n_samples=len(believe), random_state=27)
neutral_upsampled = resample(neutral, replace=True, n_samples=len(believe), random_state=27)
news_upsampled = resample(news, replace=True, n_samples=len(believe), random_state=27)


Combine majority and upsampled minority

In [6]:
df_upsampled = pd.concat([believe, no_belief_upsampled, neutral_upsampled, news_upsampled])


In [7]:
df_upsampled.sentiment.value_counts()

-1    8530
 2    8530
 1    8530
 0    8530
Name: sentiment, dtype: int64

In [8]:
df_upsampled

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954
5,1,Worth a read whether you do or don't believe i...,425577
...,...,...,...
15472,2,RT @ABC7: President Trump signs executive orde...,914103
9341,2,RT @RoseBartu: Jane Goodall calls Trump's clim...,44676
10506,2,Government to outline climate change risks fac...,321347
12624,2,Teens suing US over climate change ask for Exx...,247796


Splitting out the X variable from the target

In [9]:
y = df_upsampled['sentiment']
X = df_upsampled['message']

Turning text into something your model can read

In [10]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=2, stop_words="english", max_df=0.75)
X_vectorized = vectorizer.fit_transform(X)

Splitting the training data into a training and validation set

In [11]:
X_train,X_val,y_train,y_val = train_test_split(X_vectorized,y,test_size=.3,shuffle=True, stratify=y, random_state=11)

Training the model and evaluating using the validation set

In [12]:
lsvc = LinearSVC()
lsvc.fit(X_train, y_train)
lsvc_pred = lsvc.predict(X_val)

Checking the performance of our model on the validation set

In [13]:
f1_score(y_val, lsvc_pred, average="macro")

0.9570665564988128

Getting our test set ready

In [14]:
testx = test['message']
test_vect = vectorizer.transform(testx)

Making predictions on the test set and adding a sentiment column to our original test df

In [15]:
y_pred = lsvc.predict(test_vect)

In [16]:
test['sentiment'] = y_pred

In [17]:
test.head()

,message,tweetid,sentiment
0,Europe will now be looking to China to make su...,169760,1
1,Combine this with the polling of staffers re c...,35326,1
2,"The scary, unimpeachable evidence that climate...",224985,1
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263,1
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928,0


Creating an output csv for submission

In [18]:
test[['tweetid','sentiment']].to_csv('testsubmission.csv', index=False)